In [1]:
import cv2
import os
import json
import numpy as np
import pandas as pd
from PIL import Image, ImageChops, ImageEnhance, ImageDraw
from mtcnn import MTCNN

In [2]:
train_sample_metadata = pd.read_json('../input/deepfake-detection-challenge/train_sample_videos/metadata.json').T
train_sample_metadata

,label,split,original
aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
abarnvbtwb.mp4,REAL,train,None
abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4
...,...,...,...
etejaapnxh.mp4,FAKE,train,wtreibcmgm.mp4
etmcruaihe.mp4,FAKE,train,afoovlsmtx.mp4
etohcvnzbj.mp4,FAKE,train,bdnaqemxmr.mp4
eudeqjhdfd.mp4,REAL,train,None


In [3]:
train_sample_metadata.loc['etohcvnzbj.mp4']

label                 FAKE
split                train
original    bdnaqemxmr.mp4
Name: etohcvnzbj.mp4, dtype: object

In [4]:
# train 영상 path
train_sample = '../input/deepfake-detection-challenge/train_sample_videos/'
train_video_files = [train_sample + x for x in sorted(os.listdir(train_sample))]

In [5]:
train_video_files.pop()

'../input/deepfake-detection-challenge/train_sample_videos/metadata.json'

In [6]:
# original이 있는 FAKE
train_sample_metadata['FAKE has original'] = [(train_sample_metadata.iloc[:,2][i] in list(train_sample_metadata.index)) for i in train_sample_metadata.T]

In [7]:
# original이 있는 FAKE 수
len(train_sample_metadata[train_sample_metadata['FAKE has original']]['original'])

58

In [8]:
# FAKE가 있는 origin 수
len(train_sample_metadata[train_sample_metadata['FAKE has original']]['original'].unique())

42

In [9]:
train_sample_metadata['original'].value_counts()

atvmxvwyns.mp4    6
meawmsgiti.mp4    6
qeumxirsme.mp4    5
kgbkktcjxf.mp4    5
gbqrgajyca.mp4    4
                 ..
ixuouyigxa.mp4    1
qokxxuayqn.mp4    1
sunqwnmlkx.mp4    1
qjlhemtkxk.mp4    1
proiippuup.mp4    1
Name: original, Length: 209, dtype: int64

In [10]:
# FAKE가 있는 origin 영상 리스트
original_vidio_has_fake = train_sample_metadata[train_sample_metadata['FAKE has original']]['original'].unique()
original_vidio_has_fake

array(['atvmxvwyns.mp4', 'ccfoszqabv.mp4', 'dlpoieqvfb.mp4',
       'aytzyidmgs.mp4', 'dkuayagnmc.mp4', 'bzythlfnhq.mp4',
       'cppdvdejkc.mp4', 'bulkxhhknf.mp4', 'edyncaijwx.mp4',
       'dzyuwjkjui.mp4', 'efwfxwwlbw.mp4', 'cprhtltsjp.mp4',
       'djxdyjopjd.mp4', 'cpjxareypw.mp4', 'dbtbbhakdv.mp4',
       'ellavthztb.mp4', 'bxzakyopjf.mp4', 'cizlkenljw.mp4',
       'brwrlczjvi.mp4', 'drcyabprvt.mp4', 'crezycjqyk.mp4',
       'cyxlcuyznd.mp4', 'ekcrtigpab.mp4', 'ehtdtkmmli.mp4',
       'bwipwzzxxu.mp4', 'atkdltyyen.mp4', 'chtapglbcj.mp4',
       'caifxvsozs.mp4', 'bffwsjxghk.mp4', 'duycddgtrl.mp4',
       'ehccixxzoe.mp4', 'eckvhdusax.mp4', 'dakiztgtnw.mp4',
       'bgwmmujlmc.mp4', 'abarnvbtwb.mp4', 'bejhvclboh.mp4',
       'avmjormvsx.mp4', 'egghxjjmfg.mp4', 'dbnygxtwek.mp4',
       'cmbzllswnl.mp4', 'afoovlsmtx.mp4', 'bdnaqemxmr.mp4'], dtype=object)

In [11]:
# path
original_vidio_has_fake_path = '../input/deepfake-detection-challenge/train_sample_videos/' + original_vidio_has_fake

In [12]:
# 0번 origin 영상으로 만든 fake 영상들 => 6개
train_sample_metadata[train_sample_metadata['original']==original_vidio_has_fake[0]].index

Index(['abofeumbvv.mp4', 'bqkdbcqjvb.mp4', 'cdyakrxkia.mp4', 'cycacemkmt.mp4',
       'czmqpxrqoh.mp4', 'dakqwktlbi.mp4'],
      dtype='object')

In [13]:
# 경로 만들어주기 (kaggle kernel에서 필요)
try:
    os.makedirs('../working/dataset/FAKE3')
except:
    pass
try:
    os.makedirs('../working/dataset/REAL3')
except:
    pass

In [14]:
import matplotlib.pyplot as plt
%matplotlib inline 

In [15]:
# origin 영상들로 학습시작
#detector = dlib.get_frontal_face_detector()
train_video_files = sorted(original_vidio_has_fake)

#train_video_files = ['bdnaqemxmr.mp4']

vid_num = 0
all_vid = len(train_video_files)
# origin 영상 얼굴찾기
for vid in train_video_files:
    count = 0
    path = '../input/deepfake-detection-challenge/train_sample_videos/' + vid
    file_name = vid.split('.')[0]
    face_coord = {}
    cap = cv2.VideoCapture(path)
    frame = 0
    mtcnn = MTCNN() # device='cuda:0',
    
    before_face_img_coord = []
    
    while(cap.isOpened()):
        ret = cap.grab()
        if ret == False:
            break
        #if frame % 30 == 0:
        ret, image = cap.retrieve()
        #img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img = image
        face_coord[frame]=[]
        # 처음 얼굴이 나오면
        if not before_face_img_coord:
            faces = mtcnn.detect_faces(img)
            # 얼굴 한개 이상?
            for face_idx, face in enumerate(faces):
                if 'confidence' in face and face['confidence'] > 0.9:
                    x1,y1,w,h = face['box']
                    x2 = min(x1+w, img.shape[1])
                    y2 = min(y1+h, img.shape[0])
                    x1 = max(x1, 0)
                    y1 = max(y1, 0)
                    crop_img = img[y1:y2, x1:x2]
                    face_coord[frame].append([y1,y2,x1,x2])

                    #plt.imshow(crop_img)
                    #plt.show()
                    cv2.imwrite(f'../working/dataset/REAL3/REAL_{file_name}_{frame}_{face_idx}.jpg', cv2.resize(crop_img, (299, 299)))
                    before_face_img_coord.append([y1,y2,x1,x2])
                    
                    break
                    
        # 이전에 검출된 face가 있으면
        else:
            for face_idx, coord in enumerate(before_face_img_coord):
                im = img[max(coord[0]-100,0) : min(coord[1]+100,img.shape[0]),
                         max(coord[2]-100,0) : min(coord[3]+100,img.shape[1])]
                face = mtcnn.detect_faces(im)
                if face:
                    # 1개만 저장해
                    x11,y11,w,h = face[0]['box']
                    x1 = max(coord[2]-100 + x11, 0)
                    x2 = min(x1 + w, img.shape[1])
                    y1 = max(coord[0]-100 + y11, 0)
                    y2 = min(y1 + h, img.shape[0])
                    crop_img2 = img[y1:y2, x1:x2]
                    
                    
                    face_coord[frame].append([y1,y2,x1,x2])
                    before_face_img_coord[face_idx]=[y1,y2,x1,x2]
                    
                    #plt.imshow(crop_img2)
                    #plt.show()
                    cv2.imwrite(f'../working/dataset/REAL3/REAL_{file_name}_{frame}_{face_idx}.jpg', cv2.resize(crop_img2, (299, 299)))
                    crop_img = crop_img2
                else:
                    face_coord[frame].append([y1,y2,x1,x2])
                    cv2.imwrite(f'../working/dataset/REAL3/REAL_{file_name}_{frame}_{face_idx}.jpg', cv2.resize(crop_img, (299, 299)))

        frame += 1
    cap.release()
    
    # 이비디오의 fake 영상만큼 반복
    for fake_vid in train_sample_metadata[train_sample_metadata['original']==vid].index:
        fake_path = '../input/deepfake-detection-challenge/train_sample_videos/' + fake_vid
        cap2 = cv2.VideoCapture(fake_path)
        fake_vid_name = fake_vid.split('.')[0]
        frame2 = 0
        while(cap2.isOpened()):
            ret2 = cap2.grab()
            if ret2 == False:
                break
            #if frame2 % 30 == 0:
            ret2, image2 = cap2.retrieve()
            #img2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
            for face_idx, i in enumerate(face_coord[frame2]):
                y1,y2,x1,x2 = i
                crop_img3 = image2[y1:y2, x1:x2]
                cv2.imwrite(f'../working/dataset/FAKE3/FAKE_{fake_vid_name}_{frame2}_{face_idx}_{file_name}.jpg', cv2.resize(crop_img3, (299, 299)))
            frame2 += 1
        cap2.release()

        
    print(f'{vid_num}/{all_vid}')
    vid_num += 1
    

0/42
1/42
2/42
3/42
4/42
5/42
6/42
7/42
8/42
9/42
10/42
11/42
12/42
13/42
14/42
15/42
16/42
17/42
18/42
19/42
20/42
21/42
22/42
23/42
24/42
25/42
26/42
27/42
28/42
29/42
30/42
31/42
32/42
33/42
34/42
35/42
36/42
37/42


error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:4045: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [16]:
y1

-2

In [17]:
y2

148

In [18]:
y11

-2

In [19]:
coord[2]

1297

In [20]:
coord[0]

7